In [22]:
import pandas as pd
import numpy as np
    
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [23]:
def cost(y_true,y_pred ,positive_label = 'pos', negative_label='neg'):
    
    data=pd.concat([y_true,y_pred],axis=1)
    wrong_negative_labels = 0
    wrong_negative_labels_cost =0
    
    wrong_positive_labels = 0
    wrong_positive_labels_cost = 0
    
    for i,n in data.iterrows():        
        if data[data.columns[1]][i] != data[data.columns[0]][i]:
            
            if data[data.columns[1]][i] == 'pos':
                wrong_positive_labels = wrong_positive_labels+1
                wrong_positive_labels_cost = wrong_positive_labels_cost+10
                
            if data[data.columns[1]][i] == 'neg':
                wrong_negative_labels = wrong_negative_labels+1
                wrong_negative_labels_cost = wrong_negative_labels_cost+ 500
            
        
    print('wrongly labelled positives =', wrong_positive_labels)
    print('wrongly labelled negatives =', wrong_negative_labels)  
    print('total cost of wrongly labelling =' , wrong_negative_labels_cost+wrong_positive_labels_cost)
    

In [24]:
def preprocess(df):
    
    X = df.drop('class',axis=1)

    X.replace('na',np.nan,inplace=True)
    X = X[X.columns].apply(pd.to_numeric, errors='coerce')

    imp = SimpleImputer(missing_values=np.nan, strategy='mean')

    X = pd.DataFrame(imp.fit_transform(X),columns=X.columns)

    return X

In [25]:
def balance_data(df,label):

    over = SMOTE(sampling_strategy=0.3)
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    df, label = pipeline.fit_resample(df, label)
    return df, label